# QF635 BTC Future live Strategy
## BTC Future | ML by YU Lingfeng
#### reading per second data on order book information
#### training for piecewise few hours
#### live trading for targeted number of trades


In [ ]:
# Import the library notebook as a module
%run G12BTC.ipynb

#### 1.1
try:
    # Try to read the CSV file into a DataFrame
    ML_files = glob.glob('+++*.csv')    
    MLdata = pd.read_csv(ML_files[-1], index_col=[0])

except FileNotFoundError:
    # Handle the case where the file is not found
    print("File not found or unable to read the file.")

# Display the DataFrame
#     print(MLdata.shape)
#     print(MLdata.head())


models = []

# Regression
models.append(("LASSO001", Lasso(alpha = 0.01)
             )
            )

model_results_10s =\
    model_training(models, MLdata, 0.8, period=10, 
                   y_idx=1, x_idx_list=np.arange(2,MLdata.shape[1]), 
                   threshold=1)



Threshold = 6
model_choice =\
    model_results_10s[0][2]

api_key, api_secret =\
    get_credential()

size = 0.01
target_trade_num = 20
trade_num = 0
N_tier = 20
Asset = 'BTCUSDT'
api_key, api_secret =\
    get_credential()

#############################################    
print("initialize account")
stop_loss(api_key, api_secret, Asset, 3)
sleep(2)
while trade_num < target_trade_num:

    print('generate signal')
    # Get request to get order book snapshot of BTCUSDT
    response = requests.get('https://fapi.binance.com/fapi/v1/depth', params={'symbol': Asset})
    sleep(2)
    # Get json object from response
    json_object = response.json()

    # call parse() method to convert json message to order book object
    order_book = parse(json_object)
    current_order_book =\
        [
            datetime.fromtimestamp(order_book.timestamp),
            order_book.mid(),
            order_book.spread(),
        ]\
        + order_book.n_bid_ask(N_tier)

    target =\
        model_choice.predict([current_order_book[2:]])[0]
    print(f"our target is {target}")
    spread =\
        current_order_book[2]
    best_long =\
        order_book.best_ask()
    best_short =\
        order_book.best_bid() 
    print(f"mid price is {0.5*(best_short+best_short)}")
    print(f"best ask is {best_long}, best bid is {best_short}")
    print(f"spread is {spread}")
    
    recent =\
       recent_trade(api_key, api_secret, 'BTCUSDT') 
    prices = [float(entry['price']) for entry in recent[:20]]

    # Find maximum and minimum prices
    max_price = max(prices)
    min_price = min(prices)


    print("Maximum price:", max_price)
    print("Minimum price:", min_price)
    if target - best_long > Threshold * spread:
        is_buy = True
        best_long = min(min_price, best_long)   
        price_level = round(best_long*(1-0.001),1)
        TP_level = round(price_level*(1+0.0015),1)
        print(f"we shall long at {price_level}!")

    elif target - best_short < -Threshold * spread:
        is_buy = False
        best_short = max(max_price, best_short)
        price_level = round(best_short*(1+0.001),1)
        TP_level = round(price_level*(1-0.0015),1)
        print(f"we shall short at {price_level}!")

    else:
        print("we shall wait!")        
        sleep(1)
        continue

    print('send limit order')

    send_limit_order(api_key, 
                     api_secret,
                     Asset, 
                     size, 
                     is_buy, 
                     'GTC',
                     price_level)  
    
    sleep(10) 
    print('send TP order')       
    send_limit_order(api_key,
                     api_secret,
                     Asset, 
                     size, 
                     not is_buy, 
                     'GTC',
                     TP_level) 
    sleep(10)
 
    my_order =\
        get_open_orders(api_key, api_secret, Asset) 
    positions =\
        get_open_positions(api_key, api_secret, Asset)  
    sleep(2)
    
    if (len(my_order) == 0):
        if float(positions[0]['positionAmt']) == 0:
            sleep(2)
            print("we are starting over")
    
    
    while (len(my_order)>0):
        positions =\
            get_open_positions(api_key, api_secret, Asset)  
        sleep(2)
        
        if(float(positions[0]['unRealizedProfit'])<-10):
            stop_loss(api_key, api_secret, Asset, 3)
            sleep(2)
            trade_num += 1
            break        
        
        
        sleep(10)
        my_order =\
            get_open_orders(api_key, api_secret, Asset)
        sleep(2)
        if (len(my_order)==0):
            print(f'we have take profit at {TP_level}')
            trade_num += 1    

            
#############################################    
            
my_history =\
    trade_history(api_key, api_secret, Asset)

pnl = []
timetag = []
for i in range(len(my_history)):
    timetag +=\
        [datetime\
            .fromtimestamp(
            my_history[i]['time']/1000
        )]
    pnl += [float(my_history[i]['realizedPnl'])]
    
history =\
    pd.DataFrame({'time': timetag, 'pnl': pnl})
history['win'] =\
    np.sign(history['pnl'])
history.iloc[-target_trade_num:]['win'].value_counts()
history.iloc[-target_trade_num:]['pnl'].cumsum().plot()
plt.legend()
plt.title("cumulative realized PnL in USDT")

All libraries and funcitons loaded


2024-05-11 00:58:06,034 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715360286034


ML training for LASSO001 done
number of sig: 122, win rate is 63.934426229508205%
wins 78, loses 44
most accurate model is LASSO001 at 63.934426229508205% accuracy
dotenv loaded successfully: True
dotenv loaded successfully: True
initialize account
All open orders cancelled successfully.


2024-05-11 00:58:06,276 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715360286227


No open positions!
generate signal


...

2024-05-11 05:47:42,821 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715377662745
2024-05-11 05:47:54,921 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715377674921
2024-05-11 05:47:57,127 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715377677052
2024-05-11 05:48:09,230 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715377689230
2024-05-11 05:48:11,446 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715377691369
2024-05-11 05:48:23,545 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715377703545
2024-05-11 05:48:25,755 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715377705678
2024-05-11 05:48:37,855 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715377717855
2024-05-11 05:48:40,096 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715377720019
2024-05-11 05:48:52,197 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715

2024-05-11 05:57:31,926 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715378251851
2024-05-11 05:57:44,029 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715378264029
2024-05-11 05:57:46,454 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715378266375
2024-05-11 05:57:58,786 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715378278786
2024-05-11 05:58:00,990 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715378280913
2024-05-11 05:58:13,093 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715378293093
2024-05-11 05:58:15,299 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715378295222
2024-05-11 05:58:27,401 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715378307401
2024-05-11 05:58:29,604 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715378309529
2024-05-11 05:58:41,708 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715

2024-05-11 06:07:21,763 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715378841682
2024-05-11 06:07:33,869 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715378853869
2024-05-11 06:07:36,070 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715378855993
2024-05-11 06:07:48,170 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715378868170
2024-05-11 06:07:50,396 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715378870318
2024-05-11 06:08:02,532 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715378882532
2024-05-11 06:08:04,730 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715378884654
2024-05-11 06:08:16,832 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715378896832
2024-05-11 06:08:19,039 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715378898960
2024-05-11 06:08:31,139 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715

2024-05-11 06:17:10,266 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715379430186
2024-05-11 06:17:22,370 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715379442370
2024-05-11 06:17:24,577 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715379444497
2024-05-11 06:17:36,685 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715379456685
2024-05-11 06:17:38,896 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715379458817
2024-05-11 06:17:50,998 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715379470998
2024-05-11 06:17:53,267 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715379473183
2024-05-11 06:18:05,376 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715379485376
2024-05-11 06:18:07,583 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715379487503
2024-05-11 06:18:19,687 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715

2024-05-11 06:26:58,699 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715380018618
2024-05-11 06:27:10,804 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715380030804
2024-05-11 06:27:13,049 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715380032966
2024-05-11 06:27:25,155 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715380045155
2024-05-11 06:27:27,360 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715380047281
2024-05-11 06:27:39,600 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715380059600
2024-05-11 06:27:41,808 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715380061726
2024-05-11 06:27:53,926 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715380073926
2024-05-11 06:27:56,276 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715380076194
2024-05-11 06:28:08,384 [MainThread  ] [INFO ]  Query string: symbol=BTCUSDT&timestamp=1715